In [ ]:
import h5py
from nexusformat.nexus import *
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

from utils import file_searcher, alloy_inserter

In [ ]:
sample_id, number = file_searcher("./growths/processes/")
print(sample_id)
print("Total number of files:", len(sample_id))

In [ ]:
file_path = "nexus_files/hm4721IGA.nxs"

In [ ]:
# Open your HDF5 file
with h5py.File(file_path, "r") as f:
    def print_h5_structure(name, obj):
        print(name, obj)
    
    f.visititems(print_h5_structure)  # Prints dataset structure

In [ ]:
test = nxload(file_path)
print(test.tree)

In [ ]:
with h5py.File(file_path, 'r') as f:
    sample = f['/entry/sample']
    layer_keys = sorted(
        [k for k in sample.keys() if k.startswith("layer")],
        key=lambda x: int(x.replace("layer", ""))
    )

    layers = []
    for key in layer_keys:
        layer = sample[key]
        thickness = float(layer.get('thickness', [0])[()]) / 10.0  # Å to nm
        material = layer.get('chemical_formula', b'').asstr()[()] if 'chemical_formula' in layer else 'unknown'
        alloy = layer.get('alloy_fraction', [0])[()]
        if len(material) == 6:
            new_material = alloy_inserter(material, alloy)
        else:
            new_material = material
        layers.append((material, thickness, new_material))

# Draw Equal-Height Stack 
fig, ax = plt.subplots(figsize=(8, 12))
block_height = 1  
y = 0
total_blocks = len(layers)

color_map = {
    "GaAs": "lightpink",
    "InAs": "red",
    "InAlAs": "skyblue",
    "InGaAs": "gold",
    "AlGaAs": "deepskyblue",
    "Si": "lightgreen",
    "C": "lightgrey"
}

for i, (material, thickness, new_material) in enumerate(reversed(layers)):
    color = color_map.get(material, "grey")
    rect = patches.Rectangle((0, y), 1, block_height, facecolor=color, edgecolor='k')
    ax.add_patch(rect)

    ax.text(0.5, y + block_height / 2,
            f"{new_material} {round(thickness, 1)} nm",
            ha='center', va='center', fontsize=12)

    y += block_height

ax.set_xlim(0, 1)
ax.set_ylim(0, total_blocks)
ax.invert_yaxis()
ax.axis('off')
plt.title("Sample Visualization", pad=20)
plt.tight_layout()
plt.show()

In [ ]:
variables = ["alloy_fraction", "thickness", "growth_rate", "rotational_frequency"]  # Variables to plot

# Read all layers 
with h5py.File(file_path, 'r') as f:
    layer_names = []
    materials = []
    data = {var: [] for var in variables}  # dictionary to store variable values

    for layer_name in f['/entry/sample'].keys():
        if 'layer' in layer_name:
            group = f['/entry/sample'][layer_name]
            layer_names.append(layer_name)
            materials.append(group['chemical_formula'][()].decode())
            for var in variables:
                if var in group:
                    data[var].append(group[var][()])
                else:
                    data[var].append(None)  # handle missing variables

# Plot each variable 
for var in variables:
    y_values = data[var]

    fig, ax = plt.subplots(figsize=(14, 8))
    x = range(len(layer_names))

    bars = ax.bar(x, y_values, color='lightgrey', edgecolor='black', linewidth=0.7)

    ax.set_xlabel('Layer Number', fontsize=10)
    ax.set_ylabel(var.replace('_', ' ').capitalize(), fontsize=10)
    ax.set_title(f'{var.replace("_", " ").capitalize()} by Layer', fontsize=12)
    ax.set_xticks(x)
    ax.set_xticklabels(layer_names, rotation=45, ha='right', fontsize=7)

    # Auto-scale Y based on data
    if all(value is not None for value in y_values):
        y_max = max(y_values) * 1.2
        ax.set_ylim(0, y_max if y_max > 0 else 1)
    else:
        ax.set_ylim(0, 1)  # fallback

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(0.7)
    ax.spines['left'].set_linewidth(0.7)
    ax.grid(axis='y', linestyle='--', alpha=0.3, linewidth=0.5)

    # Annotate each bar with material
    for bar, material in zip(bars, materials):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                material, ha='center', va='bottom', fontsize=7, rotation=90)

    plt.tight_layout()
    plt.show()

In [ ]:
variables = ["alloy_fraction", "growth_rate", "rotational_frequency"]

# --- Read all layers ---
with h5py.File(file_path, 'r') as f:
    layer_names = []
    materials = []
    data = {var: [] for var in variables}

    for layer_name in f['/entry/sample'].keys():
        if 'layer' in layer_name:
            group = f['/entry/sample'][layer_name]
            layer_names.append(layer_name)
            materials.append(group['chemical_formula'][()].decode())
            for var in variables:
                if var in group:
                    data[var].append(group[var][()])
                else:
                    data[var].append(None)

# Convert thickness to nm
thickness_nm = [t * 0.1 if t is not None else 0 for t in data['thickness']]

# Compute bar left edges (start positions)
left_edges = np.cumsum([0] + thickness_nm[:-1])

# Plot each variable 
for var in variables:
    
    y_values = data[var]

    fig, ax = plt.subplots(figsize=(16, 8))

    # Plot bars with variable widths
    for left, width, height, mat in zip(left_edges, thickness_nm, y_values, materials):
        if height is not None:
            ax.bar(left, height, width=width, align='edge',
                   color='lightgrey', edgecolor='black', linewidth=0.7)
            ax.text(left + width/2, height + 0.01, mat, ha='center', va='bottom', fontsize=7, rotation=90)

    ax.set_xlabel('Cumulative Thickness (nm)', fontsize=10)
    ax.set_ylabel(var.replace('_', ' ').capitalize(), fontsize=10)
    ax.set_title(f'{var.replace("_", " ").capitalize()} by Cumulative Thickness', fontsize=12)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(0.7)
    ax.spines['left'].set_linewidth(0.7)
    ax.grid(axis='y', linestyle='--', alpha=0.3, linewidth=0.5)

    plt.tight_layout()
    plt.show()